<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/244_PredRevenue_Gap_Orchestrator_Tier2_BizLogic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Detect Zero Spend

In [ ]:
def detect_zero_spend_gap(
    customer_id: str,
    sales_records: List[Dict[str, Any]],
    current_period_weeks: int = 4
) -> Optional[Dict[str, Any]]:
    """
    Detect gap from zero spend in recent period.

    Args:
        customer_id: Customer ID
        sales_records: List of sales records
        current_period_weeks: Number of recent weeks to check

    Returns:
        Gap dictionary if zero spend detected, None otherwise
    """
    if not sales_records:
        return None

    # Sort by date
    sorted_records = sorted(
        sales_records,
        key=lambda x: x.get('week_start_date', '')
    )

    # Get recent weeks
    recent_records = sorted_records[-current_period_weeks:]

    # Check for zero spend
    zero_weeks = sum(1 for r in recent_records if r.get('weekly_spend', 0.0) == 0.0)

    if zero_weeks > 0:
        # Calculate expected revenue (baseline average)
        if len(sorted_records) >= 4:
            baseline_records = sorted_records[:4]
            expected_revenue = sum(r.get('weekly_spend', 0.0) for r in baseline_records) / len(baseline_records)
        else:
            expected_revenue = sum(r.get('weekly_spend', 0.0) for r in sorted_records) / len(sorted_records)

        current_revenue = 0.0  # Zero spend
        gap_amount = 0.0 - expected_revenue
        gap_percentage = -100.0 if expected_revenue > 0 else 0.0

        return {
            "customer_id": customer_id,
            "gap_type": "zero_spend",
            "current_revenue": current_revenue,
            "expected_revenue": round(expected_revenue, 2),
            "gap_amount": round(gap_amount, 2),
            "gap_percentage": round(gap_percentage, 2),
            "severity": "high",
            "weeks_at_risk": zero_weeks,
            "rationale": f"Customer had {zero_weeks} zero spend week(s) in recent period"
        }

    return None



This one is extremely important in the “gap detection” suite because **zero-spend is one of the strongest real-world churn indicators** across any subscription, retail loyalty, or B2B usage dataset.

---

# 🔍 **Function Purpose: `detect_zero_spend_gap`**

### **Business Meaning**

This function answers a simple but critical business question:

> **“Has this customer stopped buying recently — and is that behavior abnormal?”**

Zero spend is not just a minor drop.
It’s a **behavioral cliff**, and companies treat it as a **red alert**.

This function detects it and builds a **gap record** the orchestrator will later score and prioritize.

---

# 🧠 **Step-by-Step Logic (Explained Clearly)**

### **1. Bail out if there is no data**

```python
if not sales_records:
    return None
```

If the customer has no sales history, you can’t detect “zero spend.”

---

### **2. Sort the records chronologically**

```python
sorted_records = sorted(sales_records, key=lambda x: x.get('week_start_date', ''))
```

Everything relies on **time order**
→ baseline = earliest weeks
→ recent period = latest weeks

---

### **3. Extract the recent period (default: last 4 weeks)**

```python
recent_records = sorted_records[-current_period_weeks:]
```

This window defines **“current revenue behavior.”**

Companies often use:

* 4 weeks
* 30 days
* last billing cycle
* current quarter

The window is configurable.

---

### **4. Count how many of the recent weeks are zero**

```python
zero_weeks = sum(1 for r in recent_records if r.get('weekly_spend', 0.0) == 0.0)
```

Possible values:

* `0` → nothing wrong
* `1` → warning
* `2+` → high concern
* `4` → fully inactive

---

### **5. If ≥1 zero week → compute expected revenue**

If the customer has *any* recent zero-spend week, we treat it as a potential gap.

Then calculate expected revenue:

```python
if len(sorted_records) >= 4:
    expected_revenue = avg(first 4 weeks)
else:
    expected_revenue = avg(all historical weeks)
```

Why use the baseline average?

Because this answers:

> **“What SHOULD this customer have spent based on their historical pattern?”**

---

### **6. Build the gap dictionary**

If zero weeks exist:

* current revenue = 0
* expected revenue = historical baseline
* gap amount = expected - actual
* gap percentage = -100% (if expected > 0)

Severity is automatically **high**.

```python
return {
    "gap_type": "zero_spend",
    "gap_amount": -expected_revenue,
    "gap_percentage": -100.0,
    "severity": "high",
    ...
}
```

---

# 🧠 Why This Matters in Business

Zero-spend is **the #1 churn signal** for:

* retailers
* fitness memberships
* subscriptions
* SaaS accounts
* financial services
* loyalty programs
* telecom usage
* even B2B vendors

It often predicts churn **2–6 weeks in advance**, which is plenty of time to intervene.

Companies use it to trigger:

* win-back campaigns
* coupon drops
* account manager outreach
* NPS surveys
* product recommendations
* “We miss you” nudges

It’s incredibly actionable.

---

# 🤖 Why This Is Powerful for Agent Architecture

From an agent-engineering perspective, this function teaches **two essential patterns**:

---

## **1. Detect simple, high-signal behaviors**

Rule-based checks like:

* “zero spend”
* “no logins”
* “no usage”
* “no orders”
* “no clicks”
* “no support activity”

These detections are cheap, reliable, and low-ambiguity.

---

## **2. The output is a standardized “gap event”**

Notice what the function returns:

```python
{
    "gap_type": ...,
    "gap_amount": ...,
    "gap_percentage": ...,
    "severity": ...,
    ...
}
```

This is EXACTLY what the orchestrator needs later for:

* scoring
* ranking
* action recommendation
* report building

This is why your orchestrator is so clean:
Each detector emits the same “shape” of event.

---

## **3. Fits seamlessly into the agent → detection → scoring → action pipeline**

Your entire architecture follows this flow:

> **Signal → Gap → Score → Priority → Action → Report**

Zero spend detection fits that perfectly.

---

# ✔️ **Summary (Short Version)**

The `detect_zero_spend_gap` function:

* Looks at the last N weeks (default 4)
* Checks if spending is zero at least once
* Estimates expected revenue from baseline
* Calculates how large the “zero gap” is
* Flags it as a **high severity** revenue gap
* Returns a standardized gap object for the orchestrator pipeline

This is one of the **highest-value and most actionable detectors** in your MVP.




# Determine Severity

In [ ]:
def _determine_severity(gap_percentage: float) -> str:
    """
    Determine gap severity based on percentage.

    Args:
        gap_percentage: Gap percentage (negative value)

    Returns:
        "high", "medium", or "low"
    """
    if gap_percentage <= -30.0:
        return "high"
    elif gap_percentage <= -15.0:
        return "medium"
    else:
        return "low"

This pair of functions is incredibly important because they mark the transition from **raw signal interpretation** to **action-ready insights**, and together they illustrate two core orchestrator principles:

* **Normalization of signals (severity classification)**
* **Multi-detector aggregation (orchestration of insights)**

Let’s break them down cleanly and clearly.

---

# ✅ **1. `_determine_severity()` — Tiny Function, Enormous Importance**

### **What it does (succinctly):**

It converts a raw numeric metric (gap percentage) into a human-interpretable category:

* **≤ –30% → “high” severity**
* **≤ –15% → “medium” severity**
* **Otherwise → “low” severity**

### Why this is important:

This tiny function is doing something almost every production system needs:

### 🔹 **It normalizes raw data into business-friendly labels**

No stakeholder wants to read:

> “–26.89% deviation”

They want to read:

> “⚠️ Medium severity revenue drop”

This function performs:

* Thresholding
* Categorization
* Interpretability mapping

This also allows **thresholds to be adjusted by business teams without touching detector logic**, because the categories are decoupled from the detection rules.

---

# Detect All Gaps

In [ ]:
def detect_all_gaps_for_customer(
    customer_id: str,
    revenue_baseline: Dict[str, Any],
    sales_records: List[Dict[str, Any]],
    gap_thresholds: Dict[str, Any]
) -> List[Dict[str, Any]]:
    """
    Detect all types of gaps for a single customer.

    Args:
        customer_id: Customer ID
        revenue_baseline: Revenue baseline data
        sales_records: Sales records for customer
        gap_thresholds: Threshold configuration

    Returns:
        List of detected gaps
    """
    gaps = []

    # 1. Declining revenue gap
    declining_gap = detect_declining_revenue_gap(
        customer_id,
        revenue_baseline,
        threshold=gap_thresholds.get("declining_revenue_threshold", -15.0)
    )
    if declining_gap:
        gaps.append(declining_gap)

    # 2. Below baseline gap
    recent_avg = revenue_baseline.get("recent_weeks_avg", 0.0)
    below_baseline_gap = detect_below_baseline_gap(
        customer_id,
        revenue_baseline,
        current_revenue=recent_avg,
        threshold=gap_thresholds.get("below_baseline_threshold", -20.0)
    )
    if below_baseline_gap:
        gaps.append(below_baseline_gap)

    # 3. Zero spend gap
    zero_spend_gap = detect_zero_spend_gap(customer_id, sales_records)
    if zero_spend_gap:
        gaps.append(zero_spend_gap)

    return gaps



## ✅ **detect_all_gaps_for_customer()** — The “Mini Orchestrator”

### **What this function does:**

Runs ALL gap detectors for ONE customer, bundles results into a list.

It is the first step where the system moves from **individual signals** to a **coherent, multi-signal interpretation**.

### **In simple steps:**

#### **Step 1 — Run declining revenue check**

```python
declining_gap = detect_declining_revenue_gap(...)
```

If applicable → added to list.

#### **Step 2 — Run below-baseline check**

```python
below_baseline_gap = detect_below_baseline_gap(...)
```

If applicable → added to list.

#### **Step 3 — Run zero-spend gap check**

```python
zero_spend_gap = detect_zero_spend_gap(...)
```

If applicable → added to list.

#### **Step 4 — Return all results**

```python
return gaps
```

### **This function is critical — here’s why:**

---

# 🎯 **Key Concept: Multi-signal Detection Logic**

This is your first exposure to a powerful agent design pattern:

> **“Each detector is independent, but the orchestrator aggregates their outputs.”**

This gives the system:

### **✔ Modularity**

You can add new detectors ANY time:

* Seasonal drop detector
* Price sensitivity detector
* Inventory-linked demand drop detector
* Competitor switching detector

**…without modifying any existing code.**

Add detector → append output → done.

---

### **✔ Extensibility**

Companies LOVE this design.

Why?

Because as the business evolves:

* new behaviors emerge
* new KPIs matter
* new risk signals appear

This architecture allows the model to evolve WITH the business — not get rewritten.

---

### **✔ Multi-signal understanding**

A customer can have:

* Declining revenue
  AND
* Zero spend
  AND
* Below baseline

These compound signals indicate **urgent intervention**.

This function surfaces all active signals so downstream scoring & prioritization can weight the risk accurately.

---

# 🔥 Agent Architecture Takeaway

This function perfectly represents **agent-based modular orchestration**:

* Each function = a “skill” or “expert” (detector)
* This wrapper = the “coordinator”
* The orchestrator = the “strategist”

You’re seeing the *sub-orchestrator inside the main orchestrator*.
This layered approach is what makes industrial agent systems so powerful.

---

# 🧠 Business Logic Takeaway

From a business executive perspective, this is:

> **"Give me ALL the ways a customer is slipping, not just one."**

Because businesses make decisions based on:

* compound risk
* multi-factor decline
* early warning indicators
* combined severity

This function gathers all signals into one unified picture for that customer.

---

# 📌 In Summary

### `_determine_severity()` teaches:

* Clean classification logic
* Human-friendly interpretation
* Threshold-based decision-making

### `detect_all_gaps_for_customer()` teaches:

* Multi-detector orchestration
* Extensible, pluggable architecture
* How to merge multiple risk signals
* How agent systems coordinate sub-modules




Here are **clean, minimal stubs** that illustrate how easy it is to extend your orchestrator with new detection logic *without touching* the existing detectors.

Your current architecture is already modular by design — each detector is a small, independent function that returns either:

✅ **a gap dictionary**
or
❌ **None**

So adding new detectors is simply:

**Write new function → plug into `detect_all_gaps_for_customer()` → done.**

Below are practical, realistic examples.

---

# ✅ **1. Seasonal Drop Detector**

*Detects predictable seasonal declines using historical averages.*

```python
def detect_seasonal_drop(
    customer_id: str,
    sales_records: List[Dict[str, Any]],
    seasonal_baseline: Dict[str, float],
    threshold: float = -20.0
):
    """
    Compare current period revenue to expected seasonal baseline.
    seasonal_baseline example: {"January": 120, "February": 110, ...}
    """
    if not sales_records:
        return None

    # last week on record
    most_recent = sorted(sales_records, key=lambda x: x["week_start_date"])[-1]
    month_name = most_recent["week_start_date"].strftime("%B")

    expected = seasonal_baseline.get(month_name)
    actual = most_recent.get("weekly_spend", 0)

    if not expected:
        return None

    pct = ((actual - expected) / expected) * 100

    if pct <= threshold:
        return {
            "customer_id": customer_id,
            "gap_type": "seasonal_drop",
            "current_revenue": actual,
            "expected_revenue": expected,
            "gap_amount": round(actual - expected, 2),
            "gap_percentage": round(pct, 2),
            "severity": "medium",
            "rationale": f"Revenue {abs(pct):.1f}% below seasonal norms for {month_name}",
        }
```

---

# ✅ **2. Price Sensitivity Detector**

*Detects if customers reduce spend after price increases.*

```python
def detect_price_sensitivity_gap(
    customer_id: str,
    sales_records: List[Dict[str, Any]],
    price_changes: Dict[str, float],  # e.g. {"item_A": +10%}
    threshold: float = -10.0
):
    """
    Detects if spend dropped after known price increases.
    """
    if not sales_records:
        return None

    # Example logic: compare spend before vs after price change
    # (In a real system, you'd align purchases by product)
    sorted_records = sorted(sales_records, key=lambda x: x['week_start_date'])
    mid = len(sorted_records) // 2

    before = sum(r['weekly_spend'] for r in sorted_records[:mid]) / max(mid, 1)
    after = sum(r['weekly_spend'] for r in sorted_records[mid:]) / max(len(sorted_records)-mid, 1)

    pct = ((after - before) / before) * 100 if before > 0 else 0

    if pct <= threshold:
        return {
            "customer_id": customer_id,
            "gap_type": "price_sensitivity",
            "current_revenue": after,
            "expected_revenue": before,
            "gap_amount": round(after - before, 2),
            "gap_percentage": round(pct, 2),
            "severity": "medium",
            "rationale": "Customer reduced spend following price increases",
        }
```

---

# ✅ **3. Inventory-Linked Demand Drop Detector**

*Detects if revenue drop correlates with stock-outs.*

```python
def detect_inventory_gap(
    customer_id: str,
    sales_records: List[Dict[str, Any]],
    stock_data: List[Dict[str, Any]],
):
    """
    Detect demand drops caused by stock outages.
    """
    if not sales_records or not stock_data:
        return None

    # Find most recent week
    recent_week = sorted(sales_records, key=lambda x: x['week_start_date'])[-1]['week_start_date']

    # Did customer’s store have stock-out this week?
    stockout = any(
        s['week_start_date'] == recent_week and s['stock_rate'] < 0.5
        for s in stock_data
    )

    if not stockout:
        return None

    return {
        "customer_id": customer_id,
        "gap_type": "inventory_related",
        "current_revenue": sales_records[-1]['weekly_spend'],
        "expected_revenue": None,  # could compute baseline
        "gap_amount": None,
        "gap_percentage": None,
        "severity": "low",
        "rationale": "Revenue impacted due to stock-out conditions",
    }
```

---

# ✅ **4. Competitor Switching Detector**

*Detects if a drop correlates with competitor activity.*

```python
def detect_competitor_switching(
    customer_id: str,
    sales_records: List[Dict[str, Any]],
    competitor_signals: Dict[str, float],
    threshold: float = -20.0
):
    """
    Detect competitor switching using external signals (ads, apps, traffic, etc.)
    """
    recent_avg = sum(r["weekly_spend"] for r in sales_records[-4:]) / 4
    baseline_avg = sum(r["weekly_spend"] for r in sales_records[:4]) / 4

    pct = ((recent_avg - baseline_avg) / baseline_avg) * 100 if baseline_avg > 0 else 0

    competitor_pressure = competitor_signals.get(customer_id, 0)

    if pct <= threshold and competitor_pressure > 0.7:
        return {
            "customer_id": customer_id,
            "gap_type": "competitor_switch",
            "current_revenue": recent_avg,
            "expected_revenue": baseline_avg,
            "gap_amount": round(recent_avg - baseline_avg, 2),
            "gap_percentage": round(pct, 2),
            "severity": "high",
            "rationale": "Customer may be switching due to competitor activity",
        }
```

---

# ⭐ How to plug these into `detect_all_gaps_for_customer`

It's as simple as:

```python
seasonal_gap = detect_seasonal_drop(customer_id, sales_records, seasonal_baseline)
if seasonal_gap:
    gaps.append(seasonal_gap)
```

No need to modify the rest of your architecture.

---

# 🎯 Key Lessons These Examples Teach You

### 1. **Your orchestrator is extensible by design**

You can add features without breaking existing logic.

### 2. **Each detector is independent**

Pure functions → predictable → unit testable → clear business rules.

### 3. **You can mix rule-based + ML detection**

Add an ML detector the same way:

```python
ml_gap = detect_ml_gap(customer_id, sales_records, ml_predictions)
```

### 4. **You can customize to any business without rewriting core code**

This is why rule-based + modular orchestration is so powerful for enterprise AI systems.


